## Import

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('./dataset/train.csv')

## Time Series

### day of week, day of month, weekend, month, season, quarter

In [3]:
from datetime import datetime

In [4]:
total_dates = pd.DataFrame(train.iloc[:,6:].columns, columns=['Date'])

In [5]:
total_dates['Date'] = pd.to_datetime(total_dates['Date'])

In [6]:
# total_dates['DayofWeek'] = total_dates['Date'].dt.dayofweek # day of week (0-6)
# total_dates['DayofMonth'] = total_dates['Date'].dt.day # day of month (1-)
# total_dates['Weekend'] = total_dates['Date'].dt.dayofweek // 5 # weekend
# total_dates['Month'] = total_dates['Date'].dt.month # month (1-12)
# total_dates['Season'] = total_dates['Date'].dt.month % 12 // 3 # season (0-3)
# total_dates['Quarter'] = total_dates['Date'].dt.quarter # quarter (1-4)

- minmax scaling ver

In [7]:
total_dates['DayofWeek'] = total_dates['Date'].dt.dayofweek / 6 # day of week
total_dates['DayofMonth'] = (total_dates['Date'].dt.day - 1) / (total_dates['Date'].dt.daysinmonth - 1)  # day of month
total_dates['Weekend'] = total_dates['Date'].dt.dayofweek // 5 # weekend, 0 or 1
total_dates['Month'] = (total_dates['Date'].dt.month - 1) / 11 # month
total_dates['Season'] = total_dates['Date'].dt.month % 12 // 3 /3 # season
total_dates['Quarter'] = (total_dates['Date'].dt.quarter - 1) / 3 # quarter

### Holiday

In [8]:
#공휴일
holiday = train.iloc[:1, 6:].copy()
holiday.iloc[:1,:]= 0
holiday

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
holiday.loc[:1, ['2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02',
        '2022-03-01', '2022-05-05',  '2022-05-08', '2022-06-06',
        '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11',
        '2022-09-12', '2022-10-03', '2022-10-09', '2022-10-10', 
        '2022-12-25', '2023-01-01', '2023-01-21', '2023-01-22', '2023-01-23',
        '2023-01-24', '2023-03-01']] = 1

In [10]:
total_dates['Holiday'] = holiday.T.to_numpy()

### Salary

In [11]:
#월급
salary = train.iloc[:1, 6:].copy()
salary.iloc[:1,:] = 0

In [12]:
#월급날이 주말(10일, 25일)인 경우 전에 미리 지급한다는 가정
salary.loc[:, salary.columns.str.contains('-08|-09|-10|-23|-24|-25')] = 1

In [13]:
total_dates['Salary'] = salary.T.to_numpy()

In [14]:
total_dates.head(10)

,Date,DayofWeek,DayofMonth,Weekend,Month,Season,Quarter,Holiday,Salary
0,2022-01-01,0.833333,0.000000,1,0.0,0.0,0.0,1,0
1,2022-01-02,1.000000,0.033333,1,0.0,0.0,0.0,0,0
2,2022-01-03,0.000000,0.066667,0,0.0,0.0,0.0,0,0
3,2022-01-04,0.166667,0.100000,0,0.0,0.0,0.0,0,0
4,2022-01-05,0.333333,0.133333,0,0.0,0.0,0.0,0,0
5,2022-01-06,0.500000,0.166667,0,0.0,0.0,0.0,0,0
6,2022-01-07,0.666667,0.200000,0,0.0,0.0,0.0,0,0
7,2022-01-08,0.833333,0.233333,1,0.0,0.0,0.0,0,1
8,2022-01-09,1.000000,0.266667,1,0.0,0.0,0.0,0,1
9,2022-01-10,0.000000,0.300000,0,0.0,0.0,0.0,0,1


In [15]:
total_dates_old = total_dates.copy()

### Sale Event Info

In [16]:
sales_data = pd.read_csv('./dataset/sales.csv')

In [17]:
total_sales = sales_data.drop(columns=['ID', '제품', '대분류', '중분류', '소분류', '브랜드'])
total_sales

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,22400,67200,44800,0,0,44800,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,60690,0,40460,80920,20230,20230,60690
15887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27300


In [18]:
dates = [i for i in total_sales]
dates

['2022-01-01',
 '2022-01-02',
 '2022-01-03',
 '2022-01-04',
 '2022-01-05',
 '2022-01-06',
 '2022-01-07',
 '2022-01-08',
 '2022-01-09',
 '2022-01-10',
 '2022-01-11',
 '2022-01-12',
 '2022-01-13',
 '2022-01-14',
 '2022-01-15',
 '2022-01-16',
 '2022-01-17',
 '2022-01-18',
 '2022-01-19',
 '2022-01-20',
 '2022-01-21',
 '2022-01-22',
 '2022-01-23',
 '2022-01-24',
 '2022-01-25',
 '2022-01-26',
 '2022-01-27',
 '2022-01-28',
 '2022-01-29',
 '2022-01-30',
 '2022-01-31',
 '2022-02-01',
 '2022-02-02',
 '2022-02-03',
 '2022-02-04',
 '2022-02-05',
 '2022-02-06',
 '2022-02-07',
 '2022-02-08',
 '2022-02-09',
 '2022-02-10',
 '2022-02-11',
 '2022-02-12',
 '2022-02-13',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-18',
 '2022-02-19',
 '2022-02-20',
 '2022-02-21',
 '2022-02-22',
 '2022-02-23',
 '2022-02-24',
 '2022-02-25',
 '2022-02-26',
 '2022-02-27',
 '2022-02-28',
 '2022-03-01',
 '2022-03-02',
 '2022-03-03',
 '2022-03-04',
 '2022-03-05',
 '2022-03-06',
 '2022-03-07',
 '2022-03-

- 처음에 2022년 세일 내용만 반영되어 돌아간 모델이 있어 old version을 남겨둡니다.

In [19]:
s = [1.0] # 세일 당일
m = [0.5] # 세일 전, 후

korea_sale_info_old = {'2022-10-31':m,
            '2022-11-01':s,
            '2022-11-02':s, 
            '2022-11-03':s,
            '2022-11-04':s,
            '2022-11-05':s,
            '2022-11-06':s,
            '2022-11-07':s,
            '2022-11-08':s,
            '2022-11-09':s,
            '2022-11-10':s,
            '2022-11-11':s,
            '2022-11-12':s,
            '2022-11-13':s,
            '2022-11-14':s,
            '2022-11-15':s,
            '2022-11-16':m,}

emart_sale_info_old = {'2022-11-16':m,
            '2022-11-17':s,
            '2022-11-18':s, 
            '2022-11-19':s,
            '2022-11-20':s,
            '2022-11-21':s,
            '2022-11-22':s,
            '2022-11-23':s,
            '2022-11-24':s,
            '2022-11-25':s,
            '2022-11-26':s,
            '2022-11-27':s,
            '2022-11-28':s,
            '2022-11-29':s,
            '2022-11-30':s,
                '2022-12-01':m,}

redweek_sale_info_old = {'2022-05-28':m,
            '2022-05-29':s,
                '2022-05-30':m,}

olive_big_sale_info_old = {'2022-03-01':m,
                    '2022-03-02':s,
                    '2022-03-03':s,
                    '2022-03-04':s,
                    '2022-03-05':s,
                    '2022-03-06':s,
                    '2022-03-07':s,
                    '2022-03-08':s,
                    '2022-03-09':m,
                    '2022-06-01':m,
                    '2022-06-02':s,
                    '2022-06-03':s,
                    '2022-06-04':s,
                    '2022-06-05':s,
                    '2022-06-06':s,
                    '2022-06-07':s,
                    '2022-06-08':s,
                    '2022-06-09':m,
                    '2022-08-31':m,
                    '2022-09-01':s,
                    '2022-09-02':s,
                    '2022-09-03':s,
                    '2022-09-04':s,
                    '2022-09-05':s,
                    '2022-09-06':s,
                    '2022-09-07':s,
                    '2022-09-08':m,
                    '2022-11-30':m,
                    '2022-12-01':s,
                    '2022-12-02':s,
                    '2022-12-03':s,
                    '2022-12-04':s,
                    '2022-12-05':s,
                    '2022-12-06':s,
                    '2022-12-07':s,
                    '2022-12-08':m,}

olive_small_sale_info_old = {'2022-01-24':m,
                    '2022-01-25':s,
                    '2022-01-26':s,
                    '2022-01-27':s,
                    '2022-01-28':m,
                    '2022-02-24':m,
                    '2022-02-25':s,
                    '2022-02-26':s,
                    '2022-02-27':s,
                    '2022-02-28':m,
                    '2022-03-24':m,
                    '2022-03-25':s,
                    '2022-03-26':s,
                    '2022-03-27':s,
                    '2022-03-28':m,
                    '2022-04-24':m,
                    '2022-04-25':s,
                    '2022-04-26':s,
                    '2022-04-27':s,
                    '2022-04-28':m,
                    '2022-05-24':m,
                    '2022-05-25':s,
                    '2022-05-26':s,
                    '2022-05-27':s,
                    '2022-05-28':m,
                    '2022-06-24':m,
                    '2022-06-25':s,
                    '2022-06-26':s,
                    '2022-06-27':s,
                    '2022-06-28':m,
                    '2022-07-24':m,
                    '2022-07-25':s,
                    '2022-07-26':s,
                    '2022-07-27':s,
                    '2022-07-28':m,
                    '2022-08-24':m,
                    '2022-08-25':s,
                    '2022-08-26':s,
                    '2022-08-27':s,
                    '2022-08-28':m,
                    '2022-09-24':m,
                    '2022-09-25':s,
                    '2022-09-26':s,
                    '2022-09-27':s,
                    '2022-09-28':m,
                    '2022-10-24':m,
                    '2022-10-25':s,
                    '2022-10-26':s,
                    '2022-10-27':s,
                    '2022-10-28':m,
                    '2022-11-24':m,
                    '2022-11-25':s,
                    '2022-11-26':s,
                    '2022-11-27':s,
                    '2022-11-28':m,
                    '2022-12-24':m,
                    '2022-12-25':s,
                    '2022-12-26':s,
                    '2022-12-27':s,
                    '2022-12-28':m,}


In [20]:
korea_sale_info_old = {key: korea_sale_info_old[key] for key, value in korea_sale_info_old.items() if value == [1.0]}
emart_sale_info_old = {key: emart_sale_info_old[key] for key, value in emart_sale_info_old.items() if value == [1.0]}
redweek_sale_info_old = {key: redweek_sale_info_old[key] for key, value in redweek_sale_info_old.items() if value == [1.0]}
olive_big_sale_info_old = {key: olive_big_sale_info_old[key] for key, value in olive_big_sale_info_old.items() if value == [1.0]}
olive_small_sale_info_old = {key: olive_small_sale_info_old[key] for key, value in olive_small_sale_info_old.items() if value == [1.0]}

In [21]:
total_sale_info_old = korea_sale_info_old.keys()|emart_sale_info_old.keys()|redweek_sale_info_old.keys()|olive_big_sale_info_old.keys()|olive_small_sale_info_old.keys()

In [22]:
total_sale_old = np.array([1.0 if i in total_sale_info_old else 0.0 for i in total_sales])

In [23]:
total_dates_old['SaleInfo'] = total_sale_old

- new version

In [24]:
korea_sale_info = {'2022-10-31':m,
            '2022-11-01':s,
            '2022-11-02':s, 
            '2022-11-03':s,
            '2022-11-04':s,
            '2022-11-05':s,
            '2022-11-06':s,
            '2022-11-07':s,
            '2022-11-08':s,
            '2022-11-09':s,
            '2022-11-10':s,
            '2022-11-11':s,
            '2022-11-12':s,
            '2022-11-13':s,
            '2022-11-14':s,
            '2022-11-15':s,
            '2022-11-16':m,}

emart_sale_info = {'2022-11-23':m,
            '2022-11-24':s,
            '2022-11-25':s,
            '2022-11-26':s,
            '2022-11-27':s,
            '2022-11-28':s,
            '2022-11-29':s,
            '2022-11-30':s,
                '2022-12-01':m,}

redweek_sale_info = {'2022-05-22':m,
            '2022-05-23':s,
            '2022-05-24':s,
            '2022-05-25':s,
            '2022-05-26':s,
            '2022-05-27':s,
            '2022-05-28':s,
            '2022-05-29':s,
                '2022-05-30':m,}

olive_big_sale_info = {'2022-03-01':m,
                    '2022-03-02':s,
                    '2022-03-03':s,
                    '2022-03-04':s,
                    '2022-03-05':s,
                    '2022-03-06':s,
                    '2022-03-07':s,
                    '2022-03-08':s,
                    '2022-03-09':m,
                    '2022-06-01':m,
                    '2022-06-02':s,
                    '2022-06-03':s,
                    '2022-06-04':s,
                    '2022-06-05':s,
                    '2022-06-06':s,
                    '2022-06-07':s,
                    '2022-06-08':s,
                    '2022-06-09':m,
                    '2022-08-31':m,
                    '2022-09-01':s,
                    '2022-09-02':s,
                    '2022-09-03':s,
                    '2022-09-04':s,
                    '2022-09-05':s,
                    '2022-09-06':s,
                    '2022-09-07':s,
                    '2022-09-08':m,
                    '2022-11-30':m,
                    '2022-12-01':s,
                    '2022-12-02':s,
                    '2022-12-03':s,
                    '2022-12-04':s,
                    '2022-12-05':s,
                    '2022-12-06':s,
                    '2022-12-07':s,
                    '2022-12-08':m,
                    '2022-02-26':m,
                    '2022-02-27':s,
                    '2022-02-28':s,
                    '2022-03-01':s,
                    '2022-03-02':s,
                    '2022-03-03':s,
                    '2022-03-04':s,
                    '2022-03-05':s,
                    '2022-03-06':m,
                    }

olive_small_sale_info = {'2022-01-24':m,
                    '2022-01-25':s,
                    '2022-01-26':s,
                    '2022-01-27':s,
                    '2022-01-28':m,
                    '2022-02-24':m,
                    '2022-02-25':s,
                    '2022-02-26':s,
                    '2022-02-27':s,
                    '2022-02-28':m,
                    '2022-03-24':m,
                    '2022-03-25':s,
                    '2022-03-26':s,
                    '2022-03-27':s,
                    '2022-03-28':m,
                    '2022-04-24':m,
                    '2022-04-25':s,
                    '2022-04-26':s,
                    '2022-04-27':s,
                    '2022-04-28':m,
                    '2022-05-24':m,
                    '2022-05-25':s,
                    '2022-05-26':s,
                    '2022-05-27':s,
                    '2022-05-28':m,
                    '2022-06-24':m,
                    '2022-06-25':s,
                    '2022-06-26':s,
                    '2022-06-27':s,
                    '2022-06-28':m,
                    '2022-07-24':m,
                    '2022-07-25':s,
                    '2022-07-26':s,
                    '2022-07-27':s,
                    '2022-07-28':m,
                    '2022-08-24':m,
                    '2022-08-25':s,
                    '2022-08-26':s,
                    '2022-08-27':s,
                    '2022-08-28':m,
                    '2022-09-24':m,
                    '2022-09-25':s,
                    '2022-09-26':s,
                    '2022-09-27':s,
                    '2022-09-28':m,
                    '2022-10-24':m,
                    '2022-10-25':s,
                    '2022-10-26':s,
                    '2022-10-27':s,
                    '2022-10-28':m,
                    '2022-11-24':m,
                    '2022-11-25':s,
                    '2022-11-26':s,
                    '2022-11-27':s,
                    '2022-11-28':m,
                    '2022-12-24':m,
                    '2022-12-25':s,
                    '2022-12-26':s,
                    '2022-12-27':s,
                    '2022-12-28':m,
                    '2023-01-24':m,
                    '2023-01-25':s,
                    '2023-01-26':s,
                    '2023-01-27':s,
                    '2023-01-28':m,
                    '2023-02-24':m,
                    '2023-02-25':s,
                    '2023-02-26':s,
                    '2023-02-27':s,
                    '2023-02-28':m,
                    '2023-03-24':m,
                    '2023-03-25':s,
                    '2023-03-26':s,
                    '2023-03-27':s,
                    '2023-03-28':m,
                    }



In [25]:
korea_sale_info = {key: korea_sale_info[key] for key, value in korea_sale_info.items() if value == [1.0]}
emart_sale_info = {key: emart_sale_info[key] for key, value in emart_sale_info.items() if value == [1.0]}
redweek_sale_info = {key: redweek_sale_info[key] for key, value in redweek_sale_info.items() if value == [1.0]}
olive_big_sale_info = {key: olive_big_sale_info[key] for key, value in olive_big_sale_info.items() if value == [1.0]}
olive_small_sale_info = {key: olive_small_sale_info[key] for key, value in olive_small_sale_info.items() if value == [1.0]}

In [26]:
total_sale_info = korea_sale_info.keys()|emart_sale_info.keys()|redweek_sale_info.keys()|olive_big_sale_info.keys()|olive_small_sale_info.keys()

In [27]:
total_sale = np.array([1.0 if i in total_sale_info else 0.0 for i in total_sales])

In [28]:
total_dates['SaleInfo'] = total_sale

### Save File

In [29]:
total_dates_old

,Date,DayofWeek,DayofMonth,Weekend,Month,Season,Quarter,Holiday,Salary,SaleInfo
0,2022-01-01,0.833333,0.000000,1,0.000000,0.000000,0.000000,1,0,0.0
1,2022-01-02,1.000000,0.033333,1,0.000000,0.000000,0.000000,0,0,0.0
2,2022-01-03,0.000000,0.066667,0,0.000000,0.000000,0.000000,0,0,0.0
3,2022-01-04,0.166667,0.100000,0,0.000000,0.000000,0.000000,0,0,0.0
4,2022-01-05,0.333333,0.133333,0,0.000000,0.000000,0.000000,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
454,2023-03-31,0.666667,1.000000,0,0.181818,0.333333,0.000000,0,0,0.0
455,2023-04-01,0.833333,0.000000,1,0.272727,0.333333,0.333333,0,0,0.0
456,2023-04-02,1.000000,0.034483,1,0.272727,0.333333,0.333333,0,0,0.0
457,2023-04-03,0.000000,0.068966,0,0.272727,0.333333,0.333333,0,0,0.0


In [30]:
total_dates_old.to_csv('./dataset/total_dates_scaling_old.csv')

In [31]:
total_dates.to_csv('./dataset/total_dates_scaling.csv')